In [214]:
import math
from pprint import pprint
from math import log
import numpy as np
import sys
from typing import List, Tuple, Callable, Iterable
from functools import lru_cache, reduce, cmp_to_key
from itertools import (
    starmap,
    accumulate,
    chain,
    islice,
    product,
    tee,
    pairwise,
    combinations,
)
from collections import Counter, deque
from heapq import heapify, heappushpop, heappop, heappush
from operator import add, neg
from bisect import bisect_right
import matplotlib.pyplot as plt

import arrays
import trees
import lists
import graphs
import search
import strings
from importlib import reload

reload(arrays)
reload(trees)
reload(lists)
reload(graphs)
reload(search)
import arrays as ar
import trees as tr
import lists as ll
import graphs as g
import strings as s

print()

In [209]:
def delete_balanced(n, value: int):
    if not n:
        return None
    if value == n.data:
        if not (n.right and n.left):
            return n.left or n.right

        successor = lambda n: successor(n.left) if n.left else n
        value = n.data = successor(n.right).data

    if value < n.data:
        n.left = delete_balanced(n.left, value)
    else:
        n.right = delete_balanced(n.right, value)

    return n.balance()

In [222]:
n = tr.make_bfo("4 2 6 1 3 5 7")
n.display()

  _4_  
 /   \ 
 2   6 
/ \ / \
1 3 5 7


In [223]:
for x in (4, 1, 3, 6):
    n = n.delete_balanced(x)
n.display()

 5 
/ \
2 7


   __44(2)__   
  /         \  
43(1)     50(1)
---------------
44 43 50
---------------
